In [27]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


     ---------------------------------------- 0.0/265.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/265.8 MB ? eta -:--:--
     -------------------------------------- 0.0/265.8 MB 165.2 kB/s eta 0:26:49
     -------------------------------------- 0.0/265.8 MB 187.9 kB/s eta 0:23:35
     -------------------------------------- 0.0/265.8 MB 217.9 kB/s eta 0:20:20
     -------------------------------------- 0.1/265.8 MB 302.7 kB/s eta 0:14:38
     -------------------------------------- 0.2/265.8 MB 787.7 kB/s eta 0:05:38
     -------------------------------------- 0.2/265.8 MB 901.1 kB/s eta 0:04:55
     -------------------------------------- 0.3/265.8 MB 862.0 kB/s eta 0:05:09
     -------------------------------------- 0.3/265.8 MB 774.0 kB/s eta 0:05:44
     -------------------------------------- 0.3/265.8 MB 733.3 kB/s eta 0:06:03
     -------------------------------------- 0.4/265.8 MB 922.1 kB/s eta 0:04:48
     -------------------------------------- 0.4/265.8

In [29]:
import h2o
from h2o.estimators import (
    H2OGeneralizedLinearEstimator, 
    H2ORandomForestEstimator, 
    H2OGradientBoostingEstimator, 
    H2ONaiveBayesEstimator,
    H2OStackedEnsembleEstimator
)
from h2o.frame import H2OFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from C:\Users\vladi\anaconda3\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\vladi\AppData\Local\Temp\tmpe65jwasp
  JVM stdout: C:\Users\vladi\AppData\Local\Temp\tmpe65jwasp\h2o_vladi_started_from_python.out
  JVM stderr: C:\Users\vladi\AppData\Local\Temp\tmpe65jwasp\h2o_vladi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,2 months and 2 days
H2O_cluster_name:,H2O_from_python_vladi_d0r8ln
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.059 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [162]:
TEST_SIZE = 0.2
N_FOLDS = 5


In [164]:
# DATA LOADING AND PREPROCESSING
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [166]:
spam_data = fetch_openml(data_id=44, as_frame=True)  

# Split into features and target
X = spam_df.iloc[:, :-1]  # All columns except the last are features
y = spam_df.iloc[:, -1]   # The last column is the target (spam or not)

# Convert target to numeric
y = y.astype(int)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to H2O Frames
h2o_train = H2OFrame(pd.DataFrame(X_train).assign(label=y_train.values))
h2o_test = H2OFrame(pd.DataFrame(X_test).assign(label=y_test.values))

h2o_train['label'] = h2o_train['label'].asfactor()
h2o_test['label'] = h2o_test['label'].asfactor()

C:\Users\vladi\anaconda3\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [168]:
# BASE LEARNERS - SPECIFICATION

In [169]:
base_learners = {
    "LogisticRegression": H2OGeneralizedLinearEstimator(family="binomial", nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "RandomForest": H2ORandomForestEstimator(ntrees=50, max_depth=10, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "GradientBoosting": H2OGradientBoostingEstimator(ntrees=50, max_depth=5, nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True),
    "NaiveBayes": H2ONaiveBayesEstimator(nfolds=N_FOLDS, seed=42, keep_cross_validation_predictions=True)
}

In [170]:
# BASE LEARNERS - TRAINING & EVALUATION

In [ ]:
# Train each base learner using cross-validation
for name, learner in base_learners.items():
    print(f"Training {name} with {N_FOLDS}-fold cross-validation...")
    learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

results = {}
for name, learner in base_learners.items():
    performance = learner.model_performance(test_data=h2o_test)
    accuracy = performance.accuracy()[0][1]
    results[name] = accuracy
    print(f"{name} Accuracy (Test Set): {accuracy:.4f}")

# Print results
print("Base Learner Results:", results)

base_models = list(base_learners.values())

Training LogisticRegression with 5-fold cross-validation...
glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Training RandomForest with 5-fold cross-validation...
drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Training GradientBoosting with 5-fold cross-validation...
gbm Model Build progress: |██████████

In [ ]:
# METALEARNER - TRAINING & EVALUATION

In [ ]:
metalearner = H2OGeneralizedLinearEstimator(family="binomial")

# TRAIN
super_learner = H2OStackedEnsembleEstimator(
    base_models=base_models,
    metalearner_algorithm="glm"  # Uses Logistic Regression as the metalearner
)

print("\nTraining Super Learner...")
super_learner.train(x=list(range(X_train.shape[1])), y="label", training_frame=h2o_train)

# EVAL
super_performance = super_learner.model_performance(test_data=h2o_test)
super_accuracy = super_performance.accuracy()[0][1]
print(f"\nSuper Learner Accuracy: {super_accuracy:.4f}")


print("\nFinal Results Comparison:")
for name, acc in results.items():
    print(f"{name} Accuracy: {acc:.4f}")
print(f"Super Learner Accuracy: {super_accuracy:.4f}")
